* https://github.com/fchollet/keras/blob/master/examples/imdb_lstm.py

In [0]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import CuDNNLSTM, LSTM
from keras.datasets import imdb

In [0]:
r = np.load('seq_df.npy', allow_pickle=True)

In [0]:
output = [x[:, 0] for x in r]
output_set = set()
for x in output:
  output_set |= set(x)

In [0]:
mcc_dict = dict((x,i) for i,x in enumerate(output_set))

In [0]:
output = [[mcc_dict[y] for y in x] for x in output]

In [0]:
output2 = [x[:, 1] for x in r]

In [53]:
np.std([len(x) for x in output])

8.130029595411932

In [0]:
x_train = output[:10000]
x_test = output[10000:]

In [0]:
y = np.load('seq_y.npy', allow_pickle=True)

In [0]:
y= y=='F'

In [0]:
y_train = y[:10000]
y_test = y[10000:]

In [0]:
x_train2=output2[:10000]
x_test2 = output2[10000:]

In [0]:
maxlen = 15
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [0]:
x_train2 = sequence.pad_sequences(x_train2, maxlen=maxlen)
x_test2 = sequence.pad_sequences(x_test2, maxlen=maxlen)

In [67]:
model = Sequential()
model.add(Embedding(len(mcc_dict), 32))
model.add(CuDNNLSTM(64)) #, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

W0902 07:57:07.438769 140541347919744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0902 07:57:07.463414 140541347919744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0902 07:57:07.466971 140541347919744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0902 07:57:09.721474 140541347919744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0902 07:57:09.746324 140541347919744 deprecation_wrappe

In [94]:
from keras.models import Model
from keras.layers import Input, Reshape, concatenate
input1 = Input(shape=(None,))
input2 = Input(shape=(None,))
embd = Embedding(len(mcc_dict), 32)(input1)
reshape = Reshape((-1, 1) )(input2)
concat = concatenate([embd, reshape])
lstm_output = CuDNNLSTM(64)(concat)
model_output = Dense(1, activation='sigmoid')(lstm_output)
model2 = Model(inputs=[input1, input2], output=model_output)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  # Remove the CWD from sys.path while we load stuff.


In [0]:
model2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [107]:
model2.fit([x_train, x_train2], y_train,
          batch_size=32,
          epochs=10,
          validation_data=([x_test, x_test], y_test))

Train on 10000 samples, validate on 4349 samples
Epoch 1/10
10000/10000 [==============================] - 3s 293us/step - loss: 0.5829 - acc: 0.7106 - val_loss: 0.6803 - val_acc: 0.6045
Epoch 2/10
10000/10000 [==============================] - 3s 282us/step - loss: 0.5780 - acc: 0.7096 - val_loss: 0.6606 - val_acc: 0.6190
Epoch 3/10
10000/10000 [==============================] - 3s 287us/step - loss: 0.5740 - acc: 0.7130 - val_loss: 0.6744 - val_acc: 0.6314
Epoch 4/10
10000/10000 [==============================] - 3s 273us/step - loss: 0.5753 - acc: 0.7135 - val_loss: 0.6673 - val_acc: 0.6224
Epoch 5/10
10000/10000 [==============================] - 3s 280us/step - loss: 0.5719 - acc: 0.7171 - val_loss: 0.6692 - val_acc: 0.6178
Epoch 6/10
10000/10000 [==============================] - 3s 284us/step - loss: 0.5655 - acc: 0.7168 - val_loss: 0.6705 - val_acc: 0.6312
Epoch 7/10
10000/10000 [==============================] - 3s 278us/step - loss: 0.5620 - acc: 0.7251 - val_loss: 0.6946 - v

In [0]:
score, acc = model.evaluate(x_test, y_test, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

25000/25000 [==============================] - 12s    
Test score: 0.690729214754
Test accuracy: 0.82004
